# Christine's Code - Google Price Prediction (RNN)

In [41]:
# LSTM model in part 2
# Note - "Random Walk" theory => We actually cannot predict the exact future price, as variations of past and future and independent.
# Predict trends

# Part 1:  Import Packages

In [42]:
# Packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


# SK Learn
from sklearn.preprocessing import MinMaxScaler

In [43]:
# Import the TRAIN dataset
dataset_train = pd.read_csv('/Users/christine/VSCode/deeplearningcourse_remote/deeplearningcourse/Part 3 - Recurrent Neural Networks/Google_Stock_Price_Train.csv')

In [44]:
dataset_train.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"


In [45]:
dataset_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258 entries, 0 to 1257
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1258 non-null   object 
 1   Open    1258 non-null   float64
 2   High    1258 non-null   float64
 3   Low     1258 non-null   float64
 4   Close   1258 non-null   object 
 5   Volume  1258 non-null   object 
dtypes: float64(3), object(3)
memory usage: 59.1+ KB


In [46]:
# We are only using Open and High Column for our RNN model:
# [,1:2] - we are only taking only Open Column and we need a flatten array, easiest way is this method
# Note that it can only pass values, and we need a flatten array (n,1)
training_set = dataset_train.iloc[:, 1:2].values

training_set

array([[325.25],
       [331.27],
       [329.83],
       ...,
       [793.7 ],
       [783.33],
       [782.75]])

In [47]:
# Feature Scaling - fit and transformation later on
# But we also need to specify the transformation.

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1)) # defaults to setting 0-1 as the bounds for the values, not sure why instructor typed it.

# The scale step will fail if it's not in the (n,1) shape
training_set_scaled = sc.fit_transform(training_set)

# Refresher:
# Fit --> Calculate statistics like mean, and tell model to learn from training set
# Transform --> Apply transformations, like scaling

In [48]:
# Train Scale
training_set_scaled

#All prices are now between 0 and 1.
# Normalize - Min/Max Scaler
# Standardization - fitting like normal distribution

array([[0.08581368],
       [0.09701243],
       [0.09433366],
       ...,
       [0.95725128],
       [0.93796041],
       [0.93688146]])

In [ ]:
# Make Data Structure with 60 time-steps and 1 output
# The RNN will look at the past 60 time-steps, from T-59 up to T.
# It will use these past 60 time-steps to predict the next time-step, T+1.
# T is the current time-step, and the model predicts the value for T+1 (the next time-step).


# Part 2: Build the RNN (set up, layers, params, etc.)

# Part 3: Making predictions, visualizing results from RNN